In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/train.csv
/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/test.csv
/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/gender_submission.csv


**Tiền xử lí dữ liệu bị thiếu****

In [ ]:
Dữ liệu huấn luyện (train.csv) có:
891 hàng và 12 cột.
Cột Survived là biến mục tiêu (0: không sống sót, 1: sống sót).
Một số cột có giá trị bị thiếu:
Age: 714/891 không bị thiếu.
Cabin: chỉ có 204/891 không bị thiếu.
Embarked: 889/891 không bị thiếu.
Các bước tiếp theo:
Tiền xử lý dữ liệu:

Xử lý các giá trị bị thiếu (Age, Cabin, Embarked).
Mã hóa các cột phân loại (Sex, Embarked).
Chọn các cột quan trọng làm đặc trưng.
Xây dựng mô hình:

Dùng 5 phương pháp được chọn: Logistic Regression, KNN, Decision Tree, Random Forest, Gradient Boosting.
Chạy thực nghiệm và phân tích kết quả:

So sánh độ chính xác trên tập kiểm tra.

In [10]:
import pandas as pd

# Đọc dữ liệu
train_data = pd.read_csv("/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/train.csv")
test_data = pd.read_csv("/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/test.csv")

# Kiểm tra dữ liệu
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Load the train and test data
train_data = pd.read_csv('/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/train.csv')
test_data = pd.read_csv('/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/test.csv')

# Copy data for processing
train_processed = train_data.copy()
test_processed = test_data.copy()

# Step 1: Fill missing values
# Age: Fill missing values with the median
age_imputer = SimpleImputer(strategy='median')
train_processed['Age'] = age_imputer.fit_transform(train_processed[['Age']])
test_processed['Age'] = age_imputer.transform(test_processed[['Age']])

# Embarked: Fill missing values with the most frequent value
embarked_imputer = SimpleImputer(strategy='most_frequent')
train_processed['Embarked'] = embarked_imputer.fit_transform(train_processed[['Embarked']]).ravel()
test_processed['Embarked'] = embarked_imputer.transform(test_processed[['Embarked']]).ravel()

# Fare: Fill missing values with the median (only in test set)
fare_imputer = SimpleImputer(strategy='median')
test_processed['Fare'] = fare_imputer.fit_transform(test_processed[['Fare']])

# Step 2: Encode categorical features (Sex and Embarked)
encoder = OneHotEncoder(drop='first', sparse_output=False)  # Drop first to avoid multicollinearity
sex_encoded_train = encoder.fit_transform(train_processed[['Sex']])
sex_encoded_test = encoder.transform(test_processed[['Sex']])

embarked_encoded_train = encoder.fit_transform(train_processed[['Embarked']])
embarked_encoded_test = encoder.transform(test_processed[['Embarked']])

# Create DataFrames for encoded features
sex_encoded_train = pd.DataFrame(sex_encoded_train, columns=['Male'])
sex_encoded_test = pd.DataFrame(sex_encoded_test, columns=['Male'])

embarked_encoded_train = pd.DataFrame(embarked_encoded_train, columns=['Embarked_Q', 'Embarked_S'])
embarked_encoded_test = pd.DataFrame(embarked_encoded_test, columns=['Embarked_Q', 'Embarked_S'])

# Step 3: Combine features for training and testing
train_final = pd.concat(
    [train_processed[['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']], 
     sex_encoded_train, embarked_encoded_train],
    axis=1
)
test_final = pd.concat(
    [test_processed[['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']], 
     sex_encoded_test, embarked_encoded_test],
    axis=1
)

# Step 4: Standardize the numerical features
scaler = StandardScaler()
numerical_features_train = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
numerical_features_test = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

train_final[numerical_features_train] = scaler.fit_transform(train_final[numerical_features_train])
test_final[numerical_features_test] = scaler.transform(test_final[numerical_features_test])

print(train_final.head())
print(test_final.head())


   PassengerId  Survived    Pclass       Age     SibSp     Parch      Fare  \
0            1         0  0.827377 -0.565736  0.432793 -0.473674 -0.502445   
1            2         1 -1.566107  0.663861  0.432793 -0.473674  0.786845   
2            3         1  0.827377 -0.258337 -0.474545 -0.473674 -0.488854   
3            4         1 -1.566107  0.433312  0.432793 -0.473674  0.420730   
4            5         0  0.827377  0.433312 -0.474545 -0.473674 -0.486337   

   Male  Embarked_Q  Embarked_S  
0   1.0         0.0         1.0  
1   0.0         0.0         0.0  
2   0.0         0.0         1.0  
3   0.0         0.0         1.0  
4   1.0         0.0         1.0  
   PassengerId    Pclass       Age     SibSp     Parch      Fare  Male  \
0          892  0.827377  0.394887 -0.474545 -0.473674 -0.490783   1.0   
1          893  0.827377  1.355510  0.432793 -0.473674 -0.507479   0.0   
2          894 -0.369365  2.508257 -0.474545 -0.473674 -0.453367   1.0   
3          895  0.827377 -0.181

**1. Logistic Regression**

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Tách dữ liệu huấn luyện và kiểm tra
X_train = train_final.drop(columns=['PassengerId', 'Survived'])
y_train = train_final['Survived']
X_test = test_final.drop(columns=['PassengerId'])

# Huấn luyện mô hình
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = logreg.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_train, logreg.predict(X_train))
print(f'Accuracy: {accuracy}')

Accuracy: 0.8002244668911336


**2. K-Nearest Neighbors (KNN)**

In [18]:
from sklearn.neighbors import KNeighborsClassifier

# Huấn luyện mô hình
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = knn.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_train, knn.predict(X_train))
print(f'Accuracy: {accuracy}')

Accuracy: 0.856341189674523


**3. Decision Tree Classifier (CART)**

In [12]:
from sklearn.tree import DecisionTreeClassifier

# Huấn luyện mô hình
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = tree.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_train, tree.predict(X_train))
print(f'Accuracy: {accuracy}')

Accuracy: 0.9797979797979798


**4. Random Forest**

In [20]:
from sklearn.ensemble import RandomForestClassifier

# Huấn luyện mô hình
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = forest.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_train, forest.predict(X_train))
print(f'Accuracy: {accuracy}')

Accuracy: 0.9797979797979798


**5. Gradient Boosting (XGBoost)**

In [13]:
import xgboost as xgb
from xgboost import XGBClassifier

# Huấn luyện mô hình
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = xgb_model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_train, xgb_model.predict(X_train))
print(f'Accuracy: {accuracy}')

Accuracy: 0.9640852974186308


**khác**

In [23]:
from sklearn.model_selection import train_test_split

# Separate features (X) and target (y) from the training dataset
X = train_final.drop(columns=['PassengerId', 'Survived'])
y = train_final['Survived']

# Split into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.shape, X_val.shape, y_train.shape, y_val.shape


((712, 8), (179, 8), (712,), (179,))

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Initialize models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Train and evaluate each model
results = {}
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    # Predict on validation data
    y_pred = model.predict(X_val)
    # Calculate accuracy
    accuracy = accuracy_score(y_val, y_pred)
    results[name] = accuracy

results


{'Logistic Regression': 0.8044692737430168,
 'K-Nearest Neighbors': 0.7877094972067039,
 'Decision Tree': 0.8044692737430168,
 'Random Forest': 0.8268156424581006,
 'Gradient Boosting': 0.8044692737430168}

In [11]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Load the train and test data
train_data = pd.read_csv('/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/train.csv')
test_data = pd.read_csv('/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/test.csv')

# Copy data for processing
train_processed = train_data.copy()
test_processed = test_data.copy()

# Step 1: Fill missing values
# Age: Fill missing values with the median
age_imputer = SimpleImputer(strategy='median')
train_processed['Age'] = age_imputer.fit_transform(train_processed[['Age']])
test_processed['Age'] = age_imputer.transform(test_processed[['Age']])

# Embarked: Fill missing values with the most frequent value
embarked_imputer = SimpleImputer(strategy='most_frequent')
train_processed['Embarked'] = embarked_imputer.fit_transform(train_processed[['Embarked']]).ravel()
test_processed['Embarked'] = embarked_imputer.transform(test_processed[['Embarked']]).ravel()

# Fare: Fill missing values with the median (only in test set)
fare_imputer = SimpleImputer(strategy='median')
test_processed['Fare'] = fare_imputer.fit_transform(test_processed[['Fare']])

# Step 2: Encode categorical features (Sex and Embarked)
encoder = OneHotEncoder(drop='first', sparse_output=False)  # Drop first to avoid multicollinearity
sex_encoded_train = encoder.fit_transform(train_processed[['Sex']])
sex_encoded_test = encoder.transform(test_processed[['Sex']])

embarked_encoded_train = encoder.fit_transform(train_processed[['Embarked']])
embarked_encoded_test = encoder.transform(test_processed[['Embarked']])

# Create DataFrames for encoded features
sex_encoded_train = pd.DataFrame(sex_encoded_train, columns=['Male'])
sex_encoded_test = pd.DataFrame(sex_encoded_test, columns=['Male'])

embarked_encoded_train = pd.DataFrame(embarked_encoded_train, columns=['Embarked_Q', 'Embarked_S'])
embarked_encoded_test = pd.DataFrame(embarked_encoded_test, columns=['Embarked_Q', 'Embarked_S'])

# Step 3: Combine features for training and testing
train_final = pd.concat(
    [train_processed[['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']], 
     sex_encoded_train, embarked_encoded_train],
    axis=1
)
test_final = pd.concat(
    [test_processed[['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']], 
     sex_encoded_test, embarked_encoded_test],
    axis=1
)

# Step 4: Standardize the numerical features
scaler = StandardScaler()
numerical_features_train = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
numerical_features_test = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

train_final[numerical_features_train] = scaler.fit_transform(train_final[numerical_features_train])
test_final[numerical_features_test] = scaler.transform(test_final[numerical_features_test])

# Separate features (X) and target (y) from the training dataset
X_train = train_final.drop(columns=['PassengerId', 'Survived'])
y_train = train_final['Survived']
X_test = test_final.drop(columns=['PassengerId'])

# Initialize models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Train and evaluate each model on the training data
results = {}
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    # Predict on training data to evaluate performance
    y_pred_train = model.predict(X_train)
    # Calculate accuracy on training data
    accuracy_train = accuracy_score(y_train, y_pred_train)
    results[name] = accuracy_train

results


{'Logistic Regression': 0.8002244668911336,
 'K-Nearest Neighbors': 0.856341189674523,
 'Decision Tree': 0.9797979797979798,
 'Random Forest': 0.9797979797979798,
 'Gradient Boosting': 0.8888888888888888}

Dự đoán

In [30]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Load the train and test data
train_data = pd.read_csv('/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/train.csv')
test_data = pd.read_csv('/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/test.csv')


# Copy data for processing
train_processed = train_data.copy()
test_processed = test_data.copy()

# Step 1: Fill missing values
# Age: Fill missing values with the median
age_imputer = SimpleImputer(strategy='median')
train_processed['Age'] = age_imputer.fit_transform(train_processed[['Age']])
test_processed['Age'] = age_imputer.transform(test_processed[['Age']])

# Embarked: Fill missing values with the most frequent value
embarked_imputer = SimpleImputer(strategy='most_frequent')
train_processed['Embarked'] = embarked_imputer.fit_transform(train_processed[['Embarked']]).ravel()
test_processed['Embarked'] = embarked_imputer.transform(test_processed[['Embarked']]).ravel()

# Fare: Fill missing values with the median (only in test set)
fare_imputer = SimpleImputer(strategy='median')
test_processed['Fare'] = fare_imputer.fit_transform(test_processed[['Fare']])

# Step 2: Encode categorical features (Sex and Embarked)
encoder = OneHotEncoder(drop='first', sparse_output=False)  # Drop first to avoid multicollinearity
sex_encoded_train = encoder.fit_transform(train_processed[['Sex']])
sex_encoded_test = encoder.transform(test_processed[['Sex']])

embarked_encoded_train = encoder.fit_transform(train_processed[['Embarked']])
embarked_encoded_test = encoder.transform(test_processed[['Embarked']])

# Create DataFrames for encoded features
sex_encoded_train = pd.DataFrame(sex_encoded_train, columns=['Male'])
sex_encoded_test = pd.DataFrame(sex_encoded_test, columns=['Male'])

embarked_encoded_train = pd.DataFrame(embarked_encoded_train, columns=['Embarked_Q', 'Embarked_S'])
embarked_encoded_test = pd.DataFrame(embarked_encoded_test, columns=['Embarked_Q', 'Embarked_S'])

# Step 3: Combine features for training and testing
train_final = pd.concat(
    [train_processed[['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']], 
     sex_encoded_train, embarked_encoded_train],
    axis=1
)
test_final = pd.concat(
    [test_processed[['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']], 
     sex_encoded_test, embarked_encoded_test],
    axis=1
)

# Step 4: Standardize the numerical features
scaler = StandardScaler()
numerical_features_train = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
numerical_features_test = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

train_final[numerical_features_train] = scaler.fit_transform(train_final[numerical_features_train])
test_final[numerical_features_test] = scaler.transform(test_final[numerical_features_test])

# Separate features (X) and target (y) from the training dataset
X_train = train_final.drop(columns=['PassengerId', 'Survived'])
y_train = train_final['Survived']
X_test = test_final.drop(columns=['PassengerId'])

# Initialize models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Train and evaluate each model on the training data
results = {}
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    # Predict on training data to evaluate performance
    y_pred_train = model.predict(X_train)
    # Calculate accuracy on training data
    accuracy_train = accuracy_score(y_train, y_pred_train)
    results[name] = accuracy_train

# Dự đoán trên tập kiểm tra (test.csv) và kiểm tra kết quả dự đoán của các mô hình học máy đã huấn luyện.
predictions = {}
for name, model in models.items():
    y_pred_test = model.predict(X_test)
    predictions[name] = y_pred_test

# In kết quả dự đoán của các mô hình
for name, y_pred_test in predictions.items():
    print(f"Predictions for {name}:")
    print(y_pred_test[:10])  # In 10 kết quả đầu tiên để kiểm tra

results, predictions['Logistic Regression'][:10], predictions['K-Nearest Neighbors'][:10], predictions['Decision Tree'][:10], predictions['Random Forest'][:10], predictions['Gradient Boosting'][:10]


Predictions for Logistic Regression:
[0 0 0 0 1 0 1 0 1 0]
Predictions for K-Nearest Neighbors:
[0 0 0 1 0 0 1 0 1 0]
Predictions for Decision Tree:
[0 0 1 1 1 0 0 0 1 0]
Predictions for Random Forest:
[0 0 0 1 0 0 0 0 1 0]
Predictions for Gradient Boosting:
[0 0 0 0 0 0 0 0 1 0]


({'Logistic Regression': 0.8002244668911336,
  'K-Nearest Neighbors': 0.856341189674523,
  'Decision Tree': 0.9797979797979798,
  'Random Forest': 0.9797979797979798,
  'Gradient Boosting': 0.8888888888888888},
 array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0]),
 array([0, 0, 0, 1, 0, 0, 1, 0, 1, 0]),
 array([0, 0, 1, 1, 1, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))

**File nộp**

In [40]:
import pandas as pd

# Giả sử y_pred là kết quả dự đoán của mô hình trên tập kiểm tra
# Để minh họa, mình sẽ sử dụng dự đoán từ mô hình Logistic Regression
y_pred = predictions['Logistic Regression']

# Tạo DataFrame cho file nộp kết quả
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': y_pred
})

# Lưu file nộp kết quả
submission.to_csv('submission.csv', index=False)

# Tải file về máy tính
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv

model khác

In [8]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# Load the train and test data
train_data = pd.read_csv('/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/train.csv')
test_data = pd.read_csv('/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/test.csv')


# Feature Engineering
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch']
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch']
train_data['IsAlone'] = (train_data['FamilySize'] == 0).astype(int)
test_data['IsAlone'] = (test_data['FamilySize'] == 0).astype(int)

# Fill missing values
age_imputer = SimpleImputer(strategy='median')
train_data['Age'] = age_imputer.fit_transform(train_data[['Age']])
test_data['Age'] = age_imputer.transform(test_data[['Age']])

embarked_imputer = SimpleImputer(strategy='most_frequent')
train_data['Embarked'] = embarked_imputer.fit_transform(train_data[['Embarked']]).ravel()
test_data['Embarked'] = embarked_imputer.transform(test_data[['Embarked']]).ravel()

fare_imputer = SimpleImputer(strategy='median')
test_data['Fare'] = fare_imputer.fit_transform(test_data[['Fare']])

# Encode categorical features
encoder = OneHotEncoder(drop='first', sparse_output=False)
sex_encoded_train = encoder.fit_transform(train_data[['Sex']])
sex_encoded_test = encoder.transform(test_data[['Sex']])
embarked_encoded_train = encoder.fit_transform(train_data[['Embarked']])
embarked_encoded_test = encoder.transform(test_data[['Embarked']])

# Create DataFrames for encoded features
sex_encoded_train = pd.DataFrame(sex_encoded_train, columns=['Male'])
sex_encoded_test = pd.DataFrame(sex_encoded_test, columns=['Male'])
embarked_encoded_train = pd.DataFrame(embarked_encoded_train, columns=['Embarked_Q', 'Embarked_S'])
embarked_encoded_test = pd.DataFrame(embarked_encoded_test, columns=['Embarked_Q', 'Embarked_S'])

# Combine features for training and testing
train_final = pd.concat(
    [train_data[['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone']], 
     sex_encoded_train, embarked_encoded_train],
    axis=1
)
test_final = pd.concat(
    [test_data[['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone']], 
     sex_encoded_test, embarked_encoded_test],
    axis=1
)

# Standardize the numerical features
scaler = StandardScaler()
numerical_features_train = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize']
numerical_features_test = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize']

train_final[numerical_features_train] = scaler.fit_transform(train_final[numerical_features_train])
test_final[numerical_features_test] = scaler.transform(test_final[numerical_features_test])

# Separate features (X) and target (y) from the training dataset
X = train_final.drop(columns=['PassengerId', 'Survived'])
y = train_final['Survived']

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize and tune the model
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters and model
best_rf = grid_search.best_estimator_

# Predict on validation data
y_pred_val = best_rf.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)
print(f'Validation Accuracy: {accuracy_val}')

# Predict on test data
y_pred_test = best_rf.predict(test_final.drop(columns=['PassengerId']))

# Create submission file
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': y_pred_test
})
submission.to_csv('submission.csv', index=False)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Validation Accuracy: 0.8044692737430168


In [42]:
import pandas as pd

# Giả sử y_pred là kết quả dự đoán của mô hình trên tập kiểm tra
# Để minh họa, mình sẽ sử dụng dự đoán từ mô hình Logistic Regression
y_pred = predictions['Logistic Regression']

# Tạo DataFrame cho file nộp kết quả
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': y_pred
})

# Lưu file nộp kết quả
submission.to_csv('submission.csv', index=False)

# Tải file về máy tính
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv